In [2]:
%matplotlib inline

In [3]:
import numpy as np
from scipy import fftpack
from scipy import signal
from numericalunits import µ0, NA, kB, hbar, mm, cm, m, s, ms, us, Hz, kHz, MHz
from numericalunits import T, K, J, g, mol, A, ohm, W, N, kg, mV, V, eV, uV, mT, uT

from FID_simulation import StorageRingMagnet, FixedProbe, Noise

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm as mcolors
import matplotlib

import pickle

In [4]:
def run_sim(multipole, strength):
    external_field = StorageRingMagnet( )
    external_field.set_strength_at_1cm(multipole, strength)
    
    nmr_probe = FixedProbe(B_field=external_field, N_cells=10000)
    noise = Noise(freq_power=-1, scale_freq=3*uV)
    nmr_probe.apply_rf_field()
    times = np.linspace(0*ms, 10*ms, 10000) # 1 MSPS
    flux = nmr_probe.pickup_flux(times, mix_down=61.74*MHz) + noise(times)
    
    
    N = len(times)                # Number of samplepoints
    xf = np.linspace(0.0, 1.0/(2.0*(times[1]-times[0]))/kHz, N/2)
    fig = plt.figure()
    ffts = {}
    for win_name, window, color in [("None", 1, "k"), 
                                     ("Hamming", signal.hamming(N), "r"),
                                     ("Hann", signal.hann(N), "b"),]:
        yf = fftpack.fft(flux/uV*window)
        ffts[win_name] = yf
        plt.plot(xf, 2.0/N * np.abs(yf[:N//2])**2/np.max(2.0/N * np.abs(yf[:N//2])**2), label=win_name, marker="x", color=color)
    plt.xlabel("f / kHz")
    plt.xlim([0, 1.0/(2.0*(times[1]-times[0]))/kHz])
    plt.xlim([48, 52])
    plt.ylabel("|FFT(f)|$^2$ / max(|FFT(f)|$^2$)")
    plt.legend(title="FFT-Window")
    def strength_to_str(strength, multipole):
        str = "%.1f ppm"%(strength*1e6)
        if strength < 1e-6:
            str = "%.1f ppb"%(strength*1e9)
        
        if multipole==1: return "Dipole: %s$\cdot (1, 0, 0)^T$"%str
        if multipole==2: return "Dipole: %s$\cdot (0, 1, 0)^T$"%str
        if multipole==3: return "Dipole: %s$\cdot (0, 0, 1)^T$"%str
        
        if multipole==4: return "Quadrupole: %s/cm$\cdot (x, -y, 0)^T$"%str
        if multipole==5: return "Quadrupole: %s/cm$\cdot (z, 0, x)^T$"%str
        if multipole==6: return "Quadrupole: %s/cm$\cdot (0, -y, z)^T$"%str
        if multipole==7: return "Quadrupole: %s/cm$\cdot (y, x, 0)^T$"%str
        if multipole==8: return "Quadrupole: %s/cm$\cdot (0, z, y)^T$"%str
        
        if multipole==9: return "Sextupole: %s/cm$^2\cdot (x^2-y^2, -2xy, 0)^T$"%str
        if multipole==10: return "Sextupole: %s/cm$^2\cdot (2xz, -2yz, x^2-y^2)^T$"%str
        if multipole==11: return "Sextupole: %s/cm$^2\cdot (z^2-y^2, -2xy, 2xy)^T$"%str
        if multipole==12: return "Sextupole: %s/cm$^2\cdot (0, -2yz, z^2-y^2)^T$"%str
        if multipole==13: return "Sextupole: %s/cm$^2\cdot (2xy, x^2-y^2, 0)^T$"%str
        if multipole==14: return "Sextupole: %s/cm$^2\cdot (yz, xz, xy)^T$"%str
        if multipole==15: return "Sextupole: %s/cm$^2\cdot (0, z^2-y^2, 2yz)^T$"%str
        
        if multipole==16: return "Octupole: %s/cm$^3\cdot (x^3-3xy^2, y^3-3x^2y,0)^T$"%str
        if multipole==17: return "Octupole: %s/cm$^3\cdot (3x^2z-3zy^2, -6xyz, x^3 - 3xy^2)^T$"%str
        if multipole==18: return "Octupole: %s/cm$^3\cdot (3xz^2-3xy^2, -3x^2y-3z^2y+2y^3, 3x^2z-3zy^2)^T$"%str
        if multipole==19: return "Octupole: %s/cm$^3\cdot (z^3-3zy^2, -6xyz, 3xz^2 - 3xy^2)^T$"%str
        if multipole==20: return "Octupole: %s/cm$^3\cdot (0, y^3-3z^2y, z^3-3zy^2)^T$"%str
        if multipole==21: return "Octupole: %s/cm$^3\cdot (3x^2y-y^3, x^3-3xy^2, 0)^T$"%str
        if multipole==22: return "Octupole: %s/cm$^3\cdot (6xyz, 3x^2z-3zy^2, 3x^2y-y^3)^T$"%str
        if multipole==23: return "Octupole: %s/cm$^3\cdot (3z^2y-y^3, 3xz^2-3xy^2, 6xyz)^T$"%str
        if multipole==24: return "Octupole: %s/cm$^3\cdot (0, z^3-3zy^2, 3z^2y-y^3)^T$"%str
        
    def savepath(strength, multipole):
        if multipole >= 16:
            str = "Octopole_%d"
        elif multipole >= 9:
            str = "Sextupole_%d"
        elif multipole >= 4:
            str = "Quadrupole_%d"
        else:
            str = "Dipole_%d"
        str = str%multipole
        
        if strength < 1e-6:
            str += "_%.1f_ppb"%(strength*1e9)
        else:
            str += "_%.1f_ppm"%(strength*1e6)
        
        return str
        
    plt.title(strength_to_str(strength, multipole))
    plt.tight_layout()
    spath = savepath(strength, multipole)
    plt.savefig("./plots/"+spath+".png", dpi=200)  
    plt.savefig("./plots/"+spath+".pdf", bbox_inches="tight")  
    plt.close()
    with open("./data/"+spath+".pickle", "wb") as open_file:
        pickle.dump({"times": times, "flux": flux, "xf": xf, "ffts": ffts}, open_file)

In [5]:
for multi in range(10, 25):
    for strength in np.linspace(0, 1e-5, 11):
        run_sim(multi, strength)

c:\users\rener\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
